# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

from torch import cuda

from ate_models.ATE_BERT_Dropout_CNN_BiLSTM_Linear import ATE_BERT_Dropout_CNN_BiLSTM_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ATE_MAMS_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,iob_aspect_tags
0,The decor is not special at all but their food...,"[The, decor, is, not, special, at, all, but, t...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, ..."
1,The decor is not special at all but their food...,"[The, decor, is, not, special, at, all, but, t...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, ..."
2,The decor is not special at all but their food...,"[The, decor, is, not, special, at, all, but, t...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, ..."
3,"when tables opened up, the manager sat another...","[when, tables, opened, up, ,, the, manager, sa...","[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
4,"when tables opened up, the manager sat another...","[when, tables, opened, up, ,, the, manager, sa...","[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [10]:
SEQ_LEN = 512

In [11]:
MODEL_OUTPUT = '../../../results/ATE/MAMS/models/bert_pre_trained_dropout_cnn_bilstm_linear.pth'
STATS_OUTPUT = '../../../results/ATE/MAMS/stats/bert_pre_trained_dropout_cnn_bilstm_linear.csv'

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
best_accuracy = 0.0

In [14]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(ids_tensors[0])), 0)(ids_tensors[0])
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(tags_tensors[0])), 0)(tags_tensors[0])
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [15]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 3), tags_tensors.view(-1))
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [16]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [17]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [18]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = ATE_BERT_Dropout_CNN_BiLSTM_Linear(BertModel.from_pretrained('bert-base-uncased'), bert_seq_len=SEQ_LEN, dropout=0.3, bilstm_in_features=256, conv_out_channels=SEQ_LEN, conv_kernel_size=3, no_out_labels=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/2237, Loss: 1.1187759637832642
Epoch: 0/2, Batch: 223/2237, Loss: 0.04927690699696541
Epoch: 0/2, Batch: 446/2237, Loss: 0.047461193054914474
Epoch: 0/2, Batch: 669/2237, Loss: 0.028581561520695686
Epoch: 0/2, Batch: 892/2237, Loss: 0.049163468182086945
Epoch: 0/2, Batch: 1115/2237, Loss: 0.02915411815047264
Epoch: 0/2, Batch: 1338/2237, Loss: 0.02281624637544155
Epoch: 0/2, Batch: 1561/2237, Loss: 0.037637416273355484
Epoch: 0/2, Batch: 1784/2237, Loss: 0.026918020099401474
Epoch: 0/2, Batch: 2007/2237, Loss: 0.01651853695511818
Epoch: 0/2, Batch: 2230/2237, Loss: 0.024136990308761597
Epoch: 1/2, Batch: 0/2237, Loss: 0.03069862350821495
Epoch: 1/2, Batch: 223/2237, Loss: 0.038711726665496826
Epoch: 1/2, Batch: 446/2237, Loss: 0.024979515001177788
Epoch: 1/2, Batch: 669/2237, Loss: 0.016242971643805504
Epoch: 1/2, Batch: 892/2237, Loss: 0.03551582992076874
Epoch: 1/2, Batch: 1115/2237, Loss: 0.019024142995476723
Epoch: 1/2, Batch: 1338/2237, Loss: 0.035541

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 5/10
Epoch: 0/2, Batch: 0/2237, Loss: 1.0628331899642944
Epoch: 0/2, Batch: 223/2237, Loss: 0.05161100625991821
Epoch: 0/2, Batch: 446/2237, Loss: 0.058933358639478683
Epoch: 0/2, Batch: 669/2237, Loss: 0.03573499247431755
Epoch: 0/2, Batch: 892/2237, Loss: 0.023753123357892036
Epoch: 0/2, Batch: 1115/2237, Loss: 0.042205315083265305
Epoch: 0/2, Batch: 1338/2237, Loss: 0.022515788674354553
Epoch: 0/2, Batch: 1561/2237, Loss: 0.01568017341196537
Epoch: 0/2, Batch: 1784/2237, Loss: 0.025573570281267166
Epoch: 0/2, Batch: 2007/2237, Loss: 0.020349258556962013
Epoch: 0/2, Batch: 2230/2237, Loss: 0.03986644744873047
Epoch: 1/2, Batch: 0/2237, Loss: 0.015076952055096626
Epoch: 1/2, Batch: 223/2237, Loss: 0.051048409193754196
Epoch: 1/2, Batch: 446/2237, Loss: 0.028122583404183388
Epoch: 1/2, Batch: 669/2237, Loss: 0.03489614650607109
Epoch: 1/2, Batch: 892/2237, Loss: 0.023066947236657143
Epoch: 1/2, Batch: 1115/2237, Loss: 0.02165108732879162
Epoch: 1/2, Batch: 1338/2237, Loss: 0.016471

In [19]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.992223,0.992223,0.709229,0.992223,0.540496,0.992223,0.599163,1627.496119
1,0.991759,0.991759,0.686655,0.991759,0.477761,0.991759,0.531759,1620.617624
2,0.992184,0.992184,0.699318,0.992184,0.521614,0.992184,0.567823,1618.982137
3,0.991489,0.991489,0.516394,0.991489,0.398966,0.991489,0.429169,1633.617983
4,0.991837,0.991837,0.689122,0.991837,0.572642,0.991837,0.618884,1631.411775
5,0.992463,0.992463,0.740964,0.992463,0.569064,0.992463,0.630121,1618.451174
6,0.992406,0.992406,0.723861,0.992406,0.539544,0.992406,0.602857,1620.453216
7,0.992491,0.992491,0.723568,0.992491,0.534491,0.992491,0.593831,1644.169609
8,0.991957,0.991957,0.668212,0.991957,0.515038,0.991957,0.566107,1660.909867
9,0.992566,0.992566,0.729987,0.992566,0.564230,0.992566,0.619273,1623.852852


In [20]:
results.to_csv(STATS_OUTPUT)